In [1]:
import os
import numpy as np
import json
import pandas as pd
import itertools
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize
import re
import ast
import matplotlib.pyplot as plt

In [2]:
def create_dataframe(dataset_path):
    """Function used to create a Pandas DataFrame containing specifications page titles

    Reads products specifications from the file system ("dataset_path" variable in the main function) and creates a Pandas DataFrame where each row is a
    specification. The columns are 'source' (e.g. www.sourceA.com), 'spec_number' (e.g. 1) and the 'page title'. Note that this script will consider only
    the page title attribute for simplicity.

    Args:
        dataset_path (str): The path to the dataset

    Returns:
        df (pd.DataFrame): The Pandas DataFrame containing specifications and page titles
    """

    print('>>> Creating dataframe...\n')
    columns_df = ['source', 'spec_number', 'spec_id', 'page_title']

    progressive_id = 0
    progressive_id2row_df = {}
    for source in tqdm(os.listdir(dataset_path)):
        for specification in os.listdir(os.path.join(dataset_path, source)):
            specification_number = specification.replace('.json', '')
            specification_id = '{}//{}'.format(source, specification_number)
            with open(os.path.join(dataset_path, source, specification)) as specification_file:
                specification_data = json.load(specification_file)
                page_title = specification_data.get('<page title>').lower()
                row = (source, specification_number, specification_id, page_title)
                progressive_id2row_df.update({progressive_id: row})
                progressive_id += 1
    df = pd.DataFrame.from_dict(progressive_id2row_df, orient='index', columns=columns_df)
    print('>>> Dataframe created successfully!\n')
    return df

In [3]:
df = create_dataframe('../datasets/unlabeled/2013_camera_specs')

  0%|          | 0/24 [00:00<?, ?it/s]

>>> Creating dataframe...



100%|██████████| 24/24 [00:12<00:00,  1.87it/s]

>>> Dataframe created successfully!



## Title

In [4]:
df.head()

,source,spec_number,spec_id,page_title
0,www.wexphotographic.com,154,www.wexphotographic.com//154,nikon coolpix aw120 digital camera - camouflag...
1,www.wexphotographic.com,553,www.wexphotographic.com//553,canon ixus 150 digital camera - red (9148b007a...
2,www.wexphotographic.com,601,www.wexphotographic.com//601,fuji finepix s1 digital camera (p10nc12730a) -...
3,www.wexphotographic.com,197,www.wexphotographic.com//197,nikon coolpix s5300 digital camera - black (vn...
4,www.wexphotographic.com,178,www.wexphotographic.com//178,fuji finepix s8600 digital camera - red (p10nc...


In [5]:
df = df.drop(columns = ["source", "spec_number"], axis = 1)

In [6]:
from nltk.corpus import stopwords

stopWords = set(stopwords.words('english'))

In [7]:
stopWords = set(['itself', 'down', 'by', 'with', 'doesn', 'wouldn', 'other', 'ours', 'of', 'then', 'where', 'don', 'these', 'nor', 'she', "should've", 'won', 'ma', 'from', 'had', "you're", 'our', 'did', 'them', 'too', 'her', 'that', 'haven', 'after', "you'll", 'hers', 'because', 'yourself', 'against', 'mightn', 'as', 'll', 'whom', 'how', 'couldn', 'further', 'aren', "you'd", 'and', 'needn', "couldn't", 'those', 'to', "doesn't", "weren't", 'both', 'ourselves', 'in', 'which', 'yours', 'under', 'some', 'what', 'during', 'before', "needn't", "shan't", 'here', 'having', 'hasn', 'your', "hasn't", 'between', 'me', "she's", 'into', 'all', 'at', 'shan', 'who', 'o', 'an', 'very', 'can', 'you', 'shouldn', 'such', 'but', 'do', 'out', 'am', "shouldn't", 'above', 'wasn', 'or', 'were', 'own', 'didn', "you've", 'on', 'will', 'my', 'it', 'have', 'once', 'only', 'been', 'themselves', 'his', 'be', "mightn't", 'they', 'not', 'so', 'up', 'any', 'most', 'has', 'myself', 't', 'yourselves', 'isn', "it's", 'y', 'm', 'now', 'until', 're', 'there', 'their', 'mustn', "mustn't", 'again', 'being', 'hadn', 'doing', 'just', 'no', 'if', 've', "wasn't", "won't", 'we', 'below', 'does', 'more', 'this', 'should', "isn't", 'ain', "don't", 'i', "haven't", 'than', "didn't", 'are', 'about', 'off', 'him', 'for', 'few', "wouldn't", 'was', 'weren', 'why', 'he', "that'll", 'd', 'the', 'its', 'a', 'each', 'is', 'while', "aren't", 'when', 'theirs', 'same', 's', 'himself', 'herself', "hadn't", 'through', 'over'])

In [8]:
punctuation = "!#$%&'()*+,-./:;<=>?@[\]^_`{|}~€£¥₹₽"

In [9]:
def replace_punctuation(word):
    return ''.join(c for c in word if c not in punctuation)

In [10]:
df["page_title"] = df["page_title"].apply(lambda x : [i.lower() for i in list(map(lambda y: replace_punctuation(y), word_tokenize(x))) if i and i.lower() not in stopWords])

### Modelwords

In [11]:
pattern = re.compile("(\S*[A-Za-z]\S*[0-9]\S*|\S*[0-9]\S*[A-Za-z]\S*)")

In [12]:
## In the data replace lumix with panasonic

In [13]:
brands = ['360fly', 'acer', 'achiever', 'acorn', 'actionpro', 'activeon', 'aee', 'agfa', 'agfaphoto', 'aiptek', 'akaso', 'alpine', 'alpine', 'amkov', 'andoer', 'annke', 'ansco', 'apeman', 'apex', 'apple', 'archos', 'argus', 'arlo', 'arri', 'axis', 'bell', 'benq', 'blackmagic', 'blackmagic', 'bosch', 'bower', 'brinno', 'brookstone', 'browning', 'cambo', 'campark', 'canon', 'carl', 'casio', 'celestron', 'chinon', 'cisco', 'cobra', 'coleman', 'concord', 'contax', 'contour', 'covert', 'craig', 'crayola', 'creative', 'creative', 'crosstour', 'crumpler', 'datavideo', 'delkin', 'dell', 'digitrex', 'discovery', 'disney', 'dji', 'd-link', 'domke', 'dörr', 'dragon', 'dxg', 'dxo', 'easypix', 'elecom', 'elmo', 'emerson', 'energizer', 'epson', 'fisher-price', 'flip', 'flir', 'foscam', 'fotoman', 'fotopro', 'fuji', 'fujifilm', 'fujinon', 'garmin', 'gateway', 'godox', 'goodmans', 'google', 'gopro', 'grundig', 'hahnel', 'hamilton', 'hasselblad', 'hello', 'herofiber', 'hitachi', 'holga', 'horseman', 'hoya', 'htc', 'huawei', 'ikelite', 'ilford', 'impossible', 'innovage', 'insignia', 'insta360', 'intel', 'intova', 'ion', 'iris', 'jazz', 'jenoptik', 'jjrc', 'jvc', 'kaiser', 'kenko', 'keyence', 'king', 'kitvision', 'kodak', 'konica', 'kyocera', 'leaf', 'lego', 'leica', 'lenovo', 'lexibook', 'linhof', 'liquid', 'little', 'logitech', 'lomography', 'lowepro', 'ltl', 'lytro', 'maginon', 'magnavox', 'mamiya', 'manfrotto', 'marshall', 'marumi', 'mattel', 'maxell', 'meade', 'medion', 'memorex', 'mercury', 'metz', 'microsoft', 'microtek', 'midland', 'minolta', 'minox', 'monster', 'motorola', 'moultrie', 'mustek', 'nabi', 'neewer', 'nest', 'netgear', 'night', 'nikkon', 'nikkor', 'nikon', 'nilox', 'nintendo', 'nippon', 'nokia', 'norcent', 'olympus', 'optech', 'ordro', 'oregon', 'packard', 'palm', 'panasonic', 'parrot', 'pelco', 'pentacon', 'pentax', 'phase', 'philips', 'philips', 'phoenix', 'pioneer', 'playskool', 'polaroid', 'polarpro', 'praktica', 'premier', 'promaster', 'proscan', 'pyle', 'radioshack', 'raymarine', 'raynox', 'rca', 'ricoh', 'ring', 'rode', 'rokinon', 'rollei', 'ryobi', 'sakar', 'samsung', 'sandisk', 'sanyo', 'schneider', 'schneider', 'schneider', 'scosche', 'seasea', 'sealife', 'sharp', 'sharper', 'sigma', 'sinar', 'sipix', 'sjcam', 'sony', 'soocoo', 'stealth', 'superheadz', 'svp', 'swann', 'tamrac', 'tamron', 'technika', 'tenba', 'think', 'thule', 'tokina', 'tomy', 'toshiba', 'transcend', 'traveler', 'trust', 'verbatim', 'vibe', 'victure', 'vistaquest', 'vivitar', 'voigtländer', 'vtech', 'vupoint', 'walimex', 'wyze', 'xiaomi', 'xit', 'xtreme', 'yashica', 'zeiss']

In [14]:
df["page_title"] = df["page_title"].apply(lambda line : list(set(filter(lambda word : bool(pattern.match(word)) or word in brands,line))))

In [15]:
df.head()

,spec_id,page_title
0,www.wexphotographic.com//154,"[vna593e1, nikon, aw120]"
1,www.wexphotographic.com//553,"[canon, 9148b007aa]"
2,www.wexphotographic.com//601,"[p10nc12730a, s1, fuji]"
3,www.wexphotographic.com//197,"[s5300, nikon, vna540e1]"
4,www.wexphotographic.com//178,"[s8600, p10nc12690a, fuji]"


In [16]:
df["brand"] = [[] for _ in range(len(df))]

In [17]:
# Create brand column
for index, row in df.iterrows():
    for brand in row["page_title"]:
        if brand in brands:
            if not brand in df.at[index, "brand"]:
                df.at[index, "brand"].append(brand)
                row["page_title"].remove(brand)

In [18]:
def clean_mp_mm_g_oz(value):
    if not isinstance(value, list) and pd.isna(value):
        return np.nan
    regex = r"[0-9]+mm(\n|)"
    regex2 = r"[0-9]+mp(\n|)"
    regex3 = r"[0-9]+oz"
    regex4 = r"[0-9]+g(\n|)$"
    repl = value
    for e in repl:
        if bool(re.match(regex, e)) or bool(re.match(regex2, e)) or bool(re.match(regex3, e)) or bool(re.match(regex4, e)):
            repl.remove(e)
    return repl

In [19]:
df["page_title"] = df["page_title"].apply(lambda row : clean_mp_mm_g_oz(row))

In [20]:
df.head()

,spec_id,page_title,brand
0,www.wexphotographic.com//154,"[vna593e1, aw120]",[nikon]
1,www.wexphotographic.com//553,[9148b007aa],[canon]
2,www.wexphotographic.com//601,"[p10nc12730a, s1]",[fuji]
3,www.wexphotographic.com//197,"[s5300, vna540e1]",[nikon]
4,www.wexphotographic.com//178,"[s8600, p10nc12690a]",[fuji]


## Load cleaned datasets

In [21]:
import os
import glob

os.chdir("../datasets/unlabeled/cleaned")
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
df_cleaned = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv

In [22]:
df_cleaned = df_cleaned.reset_index(drop = True)

In [23]:
df_cleaned.head()

,spec_id,brand,weight,manufacturer,short_descr,megapixels,dimensions,screen_size,type,dots
0,www.canon-europe.com//115,canon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,www.canon-europe.com//154,canon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,www.canon-europe.com//103,canon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,www.canon-europe.com//20,canon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,www.canon-europe.com//98,canon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Merge clean with title

In [24]:
df = df.merge(df_cleaned, on="spec_id")

In [25]:
df.head()

,spec_id,page_title,brand_x,brand_y,weight,manufacturer,short_descr,megapixels,dimensions,screen_size,type,dots
0,www.wexphotographic.com//154,"[vna593e1, aw120]",[nikon],NaN,NaN,NaN,NaN,16.0,NaN,3.0,NaN,NaN
1,www.wexphotographic.com//553,[9148b007aa],[canon],NaN,NaN,NaN,NaN,16.0,NaN,2.7,NaN,NaN
2,www.wexphotographic.com//601,"[p10nc12730a, s1]",[fuji],NaN,NaN,NaN,NaN,16.4,NaN,3.0,NaN,NaN
3,www.wexphotographic.com//197,"[s5300, vna540e1]",[nikon],NaN,NaN,NaN,NaN,16.0,NaN,3.0,NaN,NaN
4,www.wexphotographic.com//178,"[s8600, p10nc12690a]",[fuji],NaN,NaN,NaN,NaN,16.0,NaN,3.0,NaN,NaN


In [26]:
df.rename(columns={"brand_x" : "brand_from_title", "brand_y" : "brand_descr"}, inplace=True)

In [27]:
df.head()

,spec_id,page_title,brand_from_title,brand_descr,weight,manufacturer,short_descr,megapixels,dimensions,screen_size,type,dots
0,www.wexphotographic.com//154,"[vna593e1, aw120]",[nikon],NaN,NaN,NaN,NaN,16.0,NaN,3.0,NaN,NaN
1,www.wexphotographic.com//553,[9148b007aa],[canon],NaN,NaN,NaN,NaN,16.0,NaN,2.7,NaN,NaN
2,www.wexphotographic.com//601,"[p10nc12730a, s1]",[fuji],NaN,NaN,NaN,NaN,16.4,NaN,3.0,NaN,NaN
3,www.wexphotographic.com//197,"[s5300, vna540e1]",[nikon],NaN,NaN,NaN,NaN,16.0,NaN,3.0,NaN,NaN
4,www.wexphotographic.com//178,"[s8600, p10nc12690a]",[fuji],NaN,NaN,NaN,NaN,16.0,NaN,3.0,NaN,NaN


In [28]:
def clean_short_descr(line):
    pattern = re.compile("(\S*[A-Za-z]\S*[0-9]\S*|\S*[0-9]\S*[A-Za-z]\S*)")
    brands = ['360fly', 'acer', 'achiever', 'acorn', 'actionpro', 'activeon', 'aee', 'agfa', 'agfaphoto', 'aiptek', 'akaso', 'alpine', 'alpine', 'amkov', 'andoer', 'annke', 'ansco', 'apeman', 'apex', 'apple', 'archos', 'argus', 'arlo', 'arri', 'axis', 'bell', 'benq', 'blackmagic', 'blackmagic', 'bosch', 'bower', 'brinno', 'brookstone', 'browning', 'cambo', 'campark', 'canon', 'carl', 'casio', 'celestron', 'chinon', 'cisco', 'cobra', 'coleman', 'concord', 'contax', 'contour', 'covert', 'craig', 'crayola', 'creative', 'creative', 'crosstour', 'crumpler', 'datavideo', 'delkin', 'dell', 'digitrex', 'discovery', 'disney', 'dji', 'd-link', 'domke', 'dörr', 'dorr', 'dragon', 'dxg', 'dxo', 'easypix', 'elecom', 'elmo', 'emerson', 'energizer', 'epson', 'fisher-price', 'flip', 'flir', 'foscam', 'fotoman', 'fotopro', 'fuji', 'fujifilm', 'fujinon', 'garmin', 'gateway', 'godox', 'goodmans', 'google', 'gopro', 'grundig', 'hahnel', 'hamilton', 'hasselblad', 'hello', 'herofiber', 'hitachi', 'holga', 'horseman', 'hoya', 'htc', 'huawei', 'ikelite', 'ilford', 'impossible', 'innovage', 'insignia', 'insta360', 'intel', 'intova', 'ion', 'iris', 'jazz', 'jenoptik', 'jjrc', 'jvc', 'kaiser', 'kenko', 'keyence', 'king', 'kitvision', 'kodak', 'konica', 'kyocera', 'leaf', 'lego', 'leica', 'lenovo', 'lexibook', 'linhof', 'liquid', 'logitech', 'lomography', 'lowepro', 'ltl', 'lytro', 'maginon', 'magnavox', 'mamiya', 'manfrotto', 'marshall', 'marumi', 'mattel', 'maxell', 'meade', 'medion', 'memorex', 'mercury', 'metz', 'microsoft', 'microtek', 'midland', 'minolta', 'minox', 'monster', 'motorola', 'moultrie', 'mustek', 'nabi', 'neewer', 'nest', 'netgear', 'nikkon', 'nikkor', 'nikon', 'nilox', 'nintendo', 'nippon', 'nokia', 'norcent', 'olympus', 'optech', 'ordro', 'oregon', 'packard', 'palm', 'panasonic', 'parrot', 'pelco', 'pentacon', 'pentax', 'phase', 'philips', 'philips', 'phoenix', 'pioneer', 'playskool', 'polaroid', 'polarpro', 'praktica', 'premier', 'promaster', 'proscan', 'pyle', 'radioshack', 'raymarine', 'raynox', 'rca', 'ricoh', 'ring', 'rode', 'rokinon', 'rollei', 'ryobi', 'sakar', 'samsung', 'sandisk', 'sanyo', 'schneider', 'schneider', 'schneider', 'scosche', 'seasea', 'sealife', 'sharp', 'sharper', 'sigma', 'sinar', 'sipix', 'sjcam', 'sony', 'soocoo', 'stealth', 'superheadz', 'svp', 'swann', 'tamrac', 'tamron', 'technika', 'tenba', 'think', 'thule', 'tokina', 'tomy', 'toshiba', 'transcend', 'traveler', 'trust', 'verbatim', 'vibe', 'victure', 'vistaquest', 'vivitar', 'voigtländer', 'vtech', 'vupoint', 'walimex', 'wyze', 'xiaomi', 'xit', 'xtreme', 'yashica', 'zeiss']
    if not isinstance(line, list) and pd.isna(line):
        return np.nan
    else:
        line = ast.literal_eval(line)
        return list(set(filter(lambda word : bool(pattern.match(word)) or word in brands,line)))

In [29]:
df["short_descr"] = df["short_descr"].apply(clean_short_descr)

In [30]:
df["short_descr"] = df["short_descr"].apply(lambda row : clean_mp_mm_g_oz(row))

In [31]:
df.head()

,spec_id,page_title,brand_from_title,brand_descr,weight,manufacturer,short_descr,megapixels,dimensions,screen_size,type,dots
0,www.wexphotographic.com//154,"[vna593e1, aw120]",[nikon],NaN,NaN,NaN,NaN,16.0,NaN,3.0,NaN,NaN
1,www.wexphotographic.com//553,[9148b007aa],[canon],NaN,NaN,NaN,NaN,16.0,NaN,2.7,NaN,NaN
2,www.wexphotographic.com//601,"[p10nc12730a, s1]",[fuji],NaN,NaN,NaN,NaN,16.4,NaN,3.0,NaN,NaN
3,www.wexphotographic.com//197,"[s5300, vna540e1]",[nikon],NaN,NaN,NaN,NaN,16.0,NaN,3.0,NaN,NaN
4,www.wexphotographic.com//178,"[s8600, p10nc12690a]",[fuji],NaN,NaN,NaN,NaN,16.0,NaN,3.0,NaN,NaN


## Add units to megapixels and screen_size

In [32]:
df["megapixels"] = df["megapixels"].apply(lambda value: str(value) + "mp" if not pd.isna(value) else np.nan)

In [33]:
df["screen_size"] = df["screen_size"].apply(lambda value: str(value) + "in" if not pd.isna(value) else np.nan)

In [34]:
df["weight"] = df["weight"].apply(lambda value: str(value) + "g" if not pd.isna(value) else np.nan)

In [35]:
df.head()

,spec_id,page_title,brand_from_title,brand_descr,weight,manufacturer,short_descr,megapixels,dimensions,screen_size,type,dots
0,www.wexphotographic.com//154,"[vna593e1, aw120]",[nikon],NaN,NaN,NaN,NaN,16.0mp,NaN,3.0in,NaN,NaN
1,www.wexphotographic.com//553,[9148b007aa],[canon],NaN,NaN,NaN,NaN,16.0mp,NaN,2.7in,NaN,NaN
2,www.wexphotographic.com//601,"[p10nc12730a, s1]",[fuji],NaN,NaN,NaN,NaN,16.4mp,NaN,3.0in,NaN,NaN
3,www.wexphotographic.com//197,"[s5300, vna540e1]",[nikon],NaN,NaN,NaN,NaN,16.0mp,NaN,3.0in,NaN,NaN
4,www.wexphotographic.com//178,"[s8600, p10nc12690a]",[fuji],NaN,NaN,NaN,NaN,16.0mp,NaN,3.0in,NaN,NaN


In [36]:
df.isna().sum() / len(df)

spec_id             0.000000
page_title          0.000000
brand_from_title    0.000000
brand_descr         0.466058
weight              0.818035
manufacturer        0.956657
short_descr         0.942120
megapixels          0.444303
dimensions          0.953837
screen_size         0.541295
type                0.951957
dots                0.981636
dtype: float64

In [37]:
len(df)

29786

In [38]:
df.head()

,spec_id,page_title,brand_from_title,brand_descr,weight,manufacturer,short_descr,megapixels,dimensions,screen_size,type,dots
0,www.wexphotographic.com//154,"[vna593e1, aw120]",[nikon],NaN,NaN,NaN,NaN,16.0mp,NaN,3.0in,NaN,NaN
1,www.wexphotographic.com//553,[9148b007aa],[canon],NaN,NaN,NaN,NaN,16.0mp,NaN,2.7in,NaN,NaN
2,www.wexphotographic.com//601,"[p10nc12730a, s1]",[fuji],NaN,NaN,NaN,NaN,16.4mp,NaN,3.0in,NaN,NaN
3,www.wexphotographic.com//197,"[s5300, vna540e1]",[nikon],NaN,NaN,NaN,NaN,16.0mp,NaN,3.0in,NaN,NaN
4,www.wexphotographic.com//178,"[s8600, p10nc12690a]",[fuji],NaN,NaN,NaN,NaN,16.0mp,NaN,3.0in,NaN,NaN


## Clean brands

In [39]:
def clean_title_brands(title):
    new_title = []
    for e in title:
        if 'fuji' in e:
            new_title.append("fuji")
        elif "nikkor" in e or "nikkon" in e:
            new_title.append("nikon")
        elif "butterfly" in e:
            new_title.append("butterfly")
        elif "blackmagic" in e:
            new_title.append("blackmagic")
        else:
            new_title.append(e)
    return list(set(new_title))

In [40]:
df["brand_from_title"] = df["brand_from_title"].apply(clean_title_brands)

In [41]:
def clean_manufacturer(e):
    if pd.isna(e):
        return e
    if 'fuji' in e:
        return "fuji"
    elif "nikkor" in e or "nikkon" in e or "niko9" in e:
        return "nikon"
    elif "penx9" in e:
        return "pentax"
    elif "canu9" in e or "canon cameras us" in e:
        return "canon"
    elif "butterfly" in e:
        return "butterfly"
    elif "blackmagic" in e:
        return "blackmagic"
    elif "leica camera" in e:
        return "leica"
    elif "samsung pleomax zirex" in e:
        return "samsung"
    elif "digital" in e or "lomo cameras" in e or "micro solution of japan" in e or "ricoh cameras usa" in e:
        np.nan
    else:
        return e

In [42]:
df["manufacturer"] = df["manufacturer"].apply(clean_manufacturer)

In [43]:
def clean_type(camera):
    if pd.isna(camera):
        return camera
    if "slr" in camera:
        return "dslr"
    elif "point shoot" in camera:
        return "point shoot"
    elif "compact" in camera:
        return "compact"
    elif "mirrorless" in camera:
        return "mirrorless"
    else:
        return camera

In [44]:
df["type"] = df["type"].apply(clean_type)

In [45]:
df["brand_descr"] = df["brand_descr"].apply(clean_manufacturer)

In [46]:
df.head()

,spec_id,page_title,brand_from_title,brand_descr,weight,manufacturer,short_descr,megapixels,dimensions,screen_size,type,dots
0,www.wexphotographic.com//154,"[vna593e1, aw120]",[nikon],NaN,NaN,NaN,NaN,16.0mp,NaN,3.0in,NaN,NaN
1,www.wexphotographic.com//553,[9148b007aa],[canon],NaN,NaN,NaN,NaN,16.0mp,NaN,2.7in,NaN,NaN
2,www.wexphotographic.com//601,"[p10nc12730a, s1]",[fuji],NaN,NaN,NaN,NaN,16.4mp,NaN,3.0in,NaN,NaN
3,www.wexphotographic.com//197,"[s5300, vna540e1]",[nikon],NaN,NaN,NaN,NaN,16.0mp,NaN,3.0in,NaN,NaN
4,www.wexphotographic.com//178,"[s8600, p10nc12690a]",[fuji],NaN,NaN,NaN,NaN,16.0mp,NaN,3.0in,NaN,NaN


## Merge brands together

In [47]:
def create_brands_column(row):
    repl = row["brand_from_title"]
    if not pd.isna(row["brand_descr"]):
        repl.append(row["brand_descr"])
    if not pd.isna(row["manufacturer"]):
        repl.append(row["manufacturer"])
    return tuple(set(repl))

In [48]:
df["merged_brands"] = df.apply(create_brands_column, axis = 1)

In [49]:
df.head()

,spec_id,page_title,brand_from_title,brand_descr,weight,manufacturer,short_descr,megapixels,dimensions,screen_size,type,dots,merged_brands
0,www.wexphotographic.com//154,"[vna593e1, aw120]",[nikon],NaN,NaN,NaN,NaN,16.0mp,NaN,3.0in,NaN,NaN,"(nikon,)"
1,www.wexphotographic.com//553,[9148b007aa],[canon],NaN,NaN,NaN,NaN,16.0mp,NaN,2.7in,NaN,NaN,"(canon,)"
2,www.wexphotographic.com//601,"[p10nc12730a, s1]",[fuji],NaN,NaN,NaN,NaN,16.4mp,NaN,3.0in,NaN,NaN,"(fuji,)"
3,www.wexphotographic.com//197,"[s5300, vna540e1]",[nikon],NaN,NaN,NaN,NaN,16.0mp,NaN,3.0in,NaN,NaN,"(nikon,)"
4,www.wexphotographic.com//178,"[s8600, p10nc12690a]",[fuji],NaN,NaN,NaN,NaN,16.0mp,NaN,3.0in,NaN,NaN,"(fuji,)"


In [50]:
sum(df.apply(lambda row : row["page_title"] == [], axis = 1))

5140

In [51]:
df.drop(columns = ["brand_from_title", "brand_descr", "manufacturer"], inplace=True)

### Statistics on matches

In [52]:
# df.head()

In [53]:
# labeled = pd.read_csv("/Users/gfotiadis/programming/sigmod/datasets/created/with_details/combined_csv.csv")

In [54]:
# labeled.head()

In [55]:
# left_joined = labeled.merge(df, left_on='left_spec_id', right_on='spec_id')
# left_joined.rename(columns={'page_title': 'left_page_title'}, inplace=True)
# left_joined.drop('spec_id', axis=1, inplace=True)

In [56]:
# left_joined.head()

In [57]:
# right_joined = labeled.merge(df, left_on='right_spec_id', right_on='spec_id')
# right_joined.rename(columns={'page_title': 'right_page_title'}, inplace=True)
# right_joined.drop('spec_id', axis=1, inplace=True)
# right_joined.head()

In [58]:
# fully_joined = pd.merge(left_joined, right_joined, how="inner", left_on=["left_spec_id", "right_spec_id"], right_on=["left_spec_id", "right_spec_id"], suffixes=("_left", "_right"))

In [59]:
# fully_joined.right_page_title.apply(lambda title : np.nan if title == [] else title).isna().sum()

In [60]:
# fully_joined.left_page_title.apply(lambda title : np.nan if title == [] else title).isna().sum()

In [61]:
# df.head()

In [62]:
grouped = df.groupby("merged_brands")

In [63]:
unbranded_til_100 = grouped.get_group(())

In [64]:
for gname, group in grouped:
    if len(group) < 100:
        unbranded_til_100 = pd.concat([group, unbranded_til_100])

In [65]:
len(unbranded_til_100)

8791

In [66]:
len(grouped.groups)

473

In [67]:
def get_merged_df(dataframe):
    merged = dataframe.drop(columns=["merged_brands"], axis = 1)
    merged = (merged.merge(merged, on=merged.assign(key_col=1)['key_col'], suffixes=('', '_right'))
 .query('spec_id < spec_id_right') # filter out joins on the same row and keep unique combinations
 .reset_index(drop=True))
    merged.drop(columns = ["key_0"], axis = 1, inplace=True)
    merged.rename(columns = {"spec_id" : "left_spec_id", "spec_id_right" : "right_spec_id"}, inplace=True)
    merged.reset_index(inplace=True)
    return merged

In [74]:
def get_merged_df_with_unbranded(dataframe_left, dataframe_right):

    merged_left = dataframe_left.drop(columns=["merged_brands"], axis = 1)
    merged_right = dataframe_right.drop(columns=["merged_brands"], axis = 1)
    
    merged_left["key_col"] = 1
    merged_right["key_col"] = 1
        
    merged = (merged_left.merge(pd.concat([merged_left, merged_right]), on='key_col', suffixes=('', '_right'))
 .query('spec_id < spec_id_right') # filter out joins on the same row and keep unique combinations
 .reset_index(drop=True))
    merged.drop(columns = ["key_col"], axis = 1, inplace=True)
    merged.rename(columns = {"spec_id" : "left_spec_id", "spec_id_right" : "right_spec_id"}, inplace=True)
    merged.reset_index(inplace=True)
    return merged

In [78]:
def determine_match(row):
    
    if row["page_title"] == [] or row["page_title_right"] == []:
        target = 0.55
    else:
        target = 1.3
            
    dim_weight = 0.9
    dots_weight = 0.9
    mp_weight = 0.44
    scr_weight = 0.54
    type_weight = 0.49
    weight_weight = 0.81
    descr_weight= 0.9
    title_weight = 0.9
    
    
    score = 0
    dim_l = row["dimensions"]
    dim_r = row["dimensions_right"]
    dots_l = row["dots"]
    dots_r = row["dots_right"]
    megapixels_l = row["megapixels"]
    megapixels_r = row["megapixels_right"]
    screen_size_l = row["screen_size"]
    screen_size_r = row["screen_size_right"]
    short_descr_l = row["short_descr"]
    short_descr_r = row["short_descr_right"]
    type_l = row["type"]
    type_r = row["type_right"]
    weight_l = row["weight"]
    weight_r = row["weight_right"]
    page_title_l = row["page_title"]
    page_title_r = row["page_title_right"]
    
    dimensions_regex = r"([0-9]+\.[0-9]+|[0-9]+)h([0-9]+\.[0-9]+|[0-9]+)w([0-9]+\.[0-9]+|[0-9]+)d"
    dimensions_regex_2 = r"h([0-9]+\.[0-9]+|[0-9]+)w([0-9]+\.[0-9]+|[0-9]+)d([0-9]+\.[0-9]+|[0-9]+)"
    
    if not pd.isna(dim_l) and not pd.isna(dim_r):
        if re.match(dimensions_regex, dim_l) == None:
            groups_l = re.match(dimensions_regex_2, dim_l).groups(1)
        else:
            groups_l = re.match(dimensions_regex, dim_l).groups(1)
        if re.match(dimensions_regex, dim_r) == None:
            groups_r = re.match(dimensions_regex_2, dim_r).groups(1)
        else:
            groups_r = re.match(dimensions_regex, dim_r).groups(1)
        if np.sum(np.abs(np.array(groups_l).astype(float) - np.array(groups_r).astype(float))) <= 0.3:
            score += dim_weight
    if not pd.isna(dots_l) and not pd.isna(dots_r) and dots_l == dots_r:
        score += dots_weight
    if not pd.isna(megapixels_l) and not pd.isna(megapixels_r) and abs(float(megapixels_l.replace("mp", "")) - float(megapixels_r.replace("mp", ""))) <= 0.2:
        score += mp_weight
    if not pd.isna(screen_size_l) and not pd.isna(screen_size_r) and abs(float(screen_size_l.replace("in", "")) - float(screen_size_r.replace("in", ""))) <= 0.2:
        score += scr_weight
    if not pd.isna(type_l) and not pd.isna(type_r) and type_l == type_r:
        score += type_weight
    if not pd.isna(weight_l) and not pd.isna(weight_r) and abs(float(weight_l.replace("g", "")) - float(weight_r.replace("g", ""))) <= 0.2:
        score += weight_weight
    
    if isinstance(short_descr_r, list) and isinstance(short_descr_l, list):
        for spec1 in short_descr_l:
            for spec2 in short_descr_r:
                if spec1 == spec2:  
                    score += descr_weight
                    
    if page_title_l != [] and page_title_r != []:
        for spec1 in page_title_l:
            for spec2 in page_title_r:
                if spec1 == spec2:  
                    score += title_weight
                    
    return score >= target
    

In [76]:
tqdm.pandas()

/Users/gfotiadis/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [79]:
for gname, group in grouped:
    labels = []
    
    if len(group) < 100 or gname == ():
        continue
        
    print("CALCULATING FOR BRAND = ", gname)

    if gname == ("canon",) or gname == ("nikon",) or gname == ("sony",):
        merged = get_merged_df_with_unbranded(group, unbranded_til_100)
    else:
        merged = get_merged_df(group)
    
    print("NUMBER OF COMPARISONS: ", len(merged))
    labels.append(list(merged.progress_apply(determine_match, axis = 1)))
    labels = sum(labels, [])
    merged["label"] = labels
    print("MATCHED ", sum(merged["label"]), " OUT OF ", len(merged["label"]))
    del labels
    merged = merged.loc[merged['label'] == True]
    cols = ["left_spec_id", "right_spec_id"]
    merged = merged[cols]
    if not '/' in gname[0]:
        merged.to_csv("/Users/gfotiadis/programming/sigmod/datasets/created/with_details/{}_matches_labeled.csv".format(gname), index = False)
    else:
        merged.to_csv("/Users/gfotiadis/programming/sigmod/datasets/created/with_details/{}_matches_labeled.csv".format(gname[0].replace("/", "")), index = False)
     

CALCULATING FOR BRAND =  ('canon',)




  0%|          | 0/14405028 [00:00<?, ?it/s]

NUMBER OF COMPARISONS:  14405028




  0%|          | 1/14405028 [00:09<37225:45:33,  9.30s/it]

  0%|          | 249/14405028 [00:09<26058:04:03,  6.51s/it]

  0%|          | 558/14405028 [00:09<18240:38:40,  4.56s/it]

  0%|          | 839/14405028 [00:09<12768:37:48,  3.19s/it]

  0%|          | 1124/14405028 [00:09<8938:17:07,  2.23s/it]

  0%|          | 1400/14405028 [00:09<6257:06:54,  1.56s/it]

  0%|          | 1702/14405028 [00:09<4380:17:13,  1.09s/it]

  0%|          | 1997/14405028 [00:10<3066:32:42,  1.30it/s]

  0%|          | 2362/14405028 [00:10<2146:51:22,  1.86it/s]

  0%|          | 2723/14405028 [00:10<1503:05:40,  2.66it/s]

  0%|          | 3039/14405028 [00:10<1052:34:23,  3.80it/s]

  0%|          | 3355/14405028 [00:10<737:09:55,  5.43it/s] 

  0%|          | 3663/14405028 [00:10<516:29:08,  7.75it/s]

  0%|          | 3954/14405028 [00:10<361:59:10, 11.05it/s]

  0%|          | 4247/14405028 [00:10<253:47:41, 15.76it/s]

  0%|          | 4568/14405028 [00:10<178:01:39, 22.47it/s]

  0%|       

KeyboardInterrupt: 



  0%|          | 33280/14405028 [00:34<1:26:22, 2773.27it/s]

In [93]:
labels_unbranded = []
merged_unbranded = get_merged_df(unbranded_til_100)
labels_unbranded.append(list(merged_unbranded.progress_apply(determine_match, axis = 1)))
labels_unbranded = sum(labels_unbranded, [])
merged_unbranded["label"] = labels_unbranded
del labels_unbranded
merged_unbranded = merged_unbranded.loc[merged_unbranded['label'] == True]
cols = ["left_spec_id", "right_spec_id"]
merged_unbranded = merged_unbranded[cols]
merged_unbranded.to_csv("/Users/gfotiadis/programming/sigmod/datasets/created/with_details/unbranded_matches_labeled.csv", index = False)

100%|██████████| 38443296/38443296 [2:07:38<00:00, 5019.94it/s]  


In [94]:
len(merged_unbranded)

169982

In [95]:
import os
import glob

os.chdir("/Users/gfotiadis/programming/sigmod/datasets/created/with_details/")
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
combined_csv = pd.concat([pd.read_csv(f, header = 0) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "/Users/gfotiadis/programming/sigmod/datasets/created/with_details/combined_csv.csv", index=False, encoding='utf-8-sig')

In [96]:
len(combined_csv)

574100

In [ ]:
# old 15100

In [ ]:
# target 600,212